# Class

In [1]:
import pandas as pd
import regex
import os
import warnings
from ast import literal_eval
from operator import itemgetter
from IPython.display import display, Markdown
from collections import Counter

class Brat_Corpus_Analysis:

    # iidcols und iloc überall, wo möglich
    # ... index_col=0) auf lange Sicht weg
    
    def __init__(self, path_annos="", path_sents="", path_data="", path_brat=""):
        
        # Paths
        self.path_annos = path_annos
        self.path_sents = path_sents
        self.path_data = "/mnt/e/Ganz_doll_wichtiges/DFKI/Masterarbeit/Datenbank/" #path_data
        self.path_brat = "/mnt/e/brat-v1.3_Crunchy_Frog/data/Masterarbeit/Analyse/" #path_brat
        
        # Input
        self.values = []  # search annotation labels per Span or per Satz
        self.searchstring = r"" # im Moment RegEx-Suche nur pro Satz
        self.brat_ges_dok = "test" # name of a brat document with multiple sentences

        # Optionen
        self.bvagheit = True # Sätze, die nur Vagheit enthalten anhand Liste herausfiltern
        self.bdateien_loeschen = False # beim Erstellen von Brat-Dateien alte löschen
        # ob pd.Series.str.contains() (benutzt re) statt regex (nötig für komplexe reg. Ausdrücke)
        self.bcontains = False 
        
        # Zeug
        self.dic_snrn_dateien = {}
        #self.START = "StartDok" # etc. -> hab aber keine Lust, dass alles umzuändern; dürfte eh immer gleich sein
        #baendern = False
        self.SNR = "SatzNr" # Name der Spalte für die Satzidentifikationsnummern
        self.identity_cols = [self.SNR,"StartDok","EndeDok"]
        
        # Dateien
        self.dfannos = pd.read_csv(self.path_annos, sep="\t")
        self.dfsaetze = pd.read_csv(self.path_sents, sep="\t", index_col=0)
        
        #self.iid_cols_saetze = [idx for idx, col in enumerate(dfalles)]
        #self.iid_cols_annos = [idx for idx, col in enumerate(dfalles)]
        
        if self.SNR not in self.dfsaetze:
            self.dfsaetze = self.dfsaetze.reset_index().rename(columns={"index":self.SNR})
        
        with open(self.path_data+"vagheitsaetze.csv", "r") as datei:
            vagheitsliste = datei.read()
        self.vagheitsliste = {int(snr) for snr in vagheitsliste.split("||")[:-1]}

    # Funktionen für Suche mit regulären Ausdrücken und regex-Modul #######################
    def regex_bool(self,value="",pattern=r""):
        if not isinstance(value, str):
            return False
        elif regex.search(pattern, value, flags=regex.IGNORECASE) is None:
            return False
        else:
            return True

    def regex_groups(self,value="",pattern=r""): # vielleicht mal verwenden
        return [group for group in regex.finditer(pattern, value, flags=regex.IGNORECASE)]
    
    # Filterung des Korpus ################################################################
    def suche(self):
        
        if not self.searchstring and not self.values:
            print("Bitte Eingabe über KSK.values = [] oder KSK.searchstring=r"" tätigen")
            return
        
        # Vorbereitung
        dfannos_fil = self.dfannos.copy() # copy() scheint nicht nötig zu sein
        self.positions = []
        self.snrn = []
        bnix = False

        """
        if baendern:
            baendern = False
        else:
            baendern = True #"""

        # Prüfen auf Rechtschreibfehler / ob was überhaupt nicht im Korpus enthalten ist 
        for group in self.values:
            for v in group:
                if v.startswith("-"):
                    v = v[1:]
                if v not in dfannos_fil["Wert"].values:
                    print("%s gar nicht gefunden"%v)

        # ggf. nach String filtern
        if self.searchstring:
            if isinstance(self.searchstring, str):
                
                # vielleicht besser das, als das direkte darunter?
                #pattern = regex.compile(self.searchstring, flags=regex.IGNORECASE) # regex_bool auch ändern
                #saetze_booleans = dfmsaetze["Satz"].apply(self.regex_bool,args=(pattern,))
                #dfalles_fil = dfalles_fil.loc[dfalles_fil["SatzNr"].isin(dfmsaetze.loc[saetze_booleans,"SatzNr"].values)]
                
                if self.bcontains:
                    warnings.filterwarnings("ignore", 'This pattern has match groups') # sinnlose Warnung abstellen
                    dfannos_fil = dfannos_fil.loc[dfannos_fil[self.SNR].isin(
                        self.dfsaetze.loc[self.dfsaetze["Satz"].str.contains(self.searchstring,),self.SNR].values)]
                else:
                    dfannos_fil = dfannos_fil.loc[dfannos_fil[self.SNR].isin(
                        self.dfsaetze.loc[self.dfsaetze["Satz"].apply(self.regex_bool,args=(self.searchstring,)),self.SNR].values)]
                
                if not self.values: # sobald auch mit searchstring Gruppierung möglich, dann das weg
                    self.positions = sorted(list({
                        (snr, std, end) for snr, std, end in dfannos_fil.loc[:,self.identity_cols].values}))
                
                # falls hier schon nichts gefunden wurde
                if dfannos_fil.shape[0] == 0:
                    print('schon mit "%s" alles rausgefiltert'%self.searchstring)
                    return

        # ggf. die Such-Werte der spans durchgehen
        for span_values in self.values:

            # die positiven Suchwerte von den negativen trennen
            # testen, ob effizienter mit for-Schleife
            values_pos = [v for v in span_values if v[0] != "-"]
            values_neg = [v[1:] for v in span_values if v[0] == "-"]

            # falls schon was gefunden wurde, nur die herausgefilterten Sätze berücksichtigen
            if self.positions:
                dfannos_fil = self.dfannos.loc[self.dfannos[self.SNR].isin({
                                                snr for snr, std, end in self.positions})]
                #print(span_values, "if self.positions:",dfannos_fil.shape)

            # Filtern mit oder() (hier mit isin)
            if values_pos and values_neg:
                dfannos_fil = dfannos_fil.loc[(~dfannos_fil["Wert"].isin(values_neg)) &
                                             (dfannos_fil["Wert"].isin(values_pos))]

            elif values_pos:
                dfannos_fil = dfannos_fil.loc[dfannos_fil["Wert"].isin(values_pos)]
                #print(span_values,"elif values_pos:",dfannos_fil.shape)

            elif values_neg:
                # enthält eine Gruppe nur Anti-Such-Werte, will ich sicherlich entsprechende Sätze ausschließen
                anti_snrn = self.dfannos.loc[self.dfannos["Wert"].isin(values_neg),self.SNR].values
                dfannos_fil = dfannos_fil.loc[~dfannos_fil[self.SNR].isin(anti_snrn)]
                
                # falls hier schon nichts mehr zu finden ist, dann abbrechen
                if not dfannos_fil.shape[0]:
                    bnix = True
                    break
                
                # die Positionen zu vergleichen ist sinnlos, wenn nur Sätze ausgeschlossen werden sollen
                self.positions = [(snr, std, end) for snr, std, end
                             in dfannos_fil.loc[:,self.identity_cols].drop_duplicates(self.identity_cols).values]
                continue

            # falls hier schon nichts mehr zu finden ist, dann abbrechen
            if not dfannos_fil.shape[0]:
                bnix = True
                break

            # ggf. Such-Gruppen, die zusammengehören sollen, filtern

            # Verschiedene Arten der Annotationen in Gruppen (DataFrames-Liste) einteilen -> ent / att / ...
            # hier könnte auch eine und()-Filterung bzgl. Annotationen ein- und desselben Typs stattfinden
            # -> z.B. [[["future","negated"],suchnomen]] -> dann in den groups-Array einfach mit normaler
            # Filterung den jeweiligen df hineinpacken zum Vergleichen
            # ähnliches System für String-Gruppen, z.B. trennbare Verben, wo z.B. "an" + PTKVZ
            dfalles_fil_grouped = dfannos_fil.groupby("Typ", sort=False)
            groups = [dfalles_fil_grouped.get_group(key).loc[:,self.identity_cols] for key in dfalles_fil_grouped.groups] #+["Typ"]
            len_groups = len(groups) - 1
            #print(span_values, "len_groups", len_groups)
            
            # die Positionen zu vergleichen ist sinnlos, wenn es keine Vergleichsgruppe gibt
            if not len_groups:
                #print(span_values, "Positionen zu vergleichen ist sinnlos, wenn es keine Vergleichsgruppe gibt")
                self.positions = [(snr, std, end) for snr, std, end
                             in dfannos_fil.loc[:,self.identity_cols].drop_duplicates(self.identity_cols).values]
                continue
    
            # groups miteinander vergleichen -> und() zw. den Gruppen, oder() innerhalb jeder group
            df_merged = dfannos_fil.loc[:,self.identity_cols].copy() # copy() nötig / besser?
            #print(span_values, "groups miteinander vergleichen", df_merged.shape)
            
            for idx_group, group in enumerate(groups):
                #print(group.iloc[0,3], idx_group, ". Gruppe", df_merged.shape) # erkannte group ausdrucken

                # nächste group mit den vorherigen vergleichen, die sich überlagernden Spans extrahieren
                df_merged = pd.\
                    merge(df_merged, group, on=self.SNR, suffixes=('', '2')).\
                    query("StartDok2 <= StartDok < EndeDok2 or StartDok2 < EndeDok <= EndeDok2 or " +
                          "StartDok <= StartDok2 < EndeDok or StartDok < EndeDok2 <= EndeDok")

                # falls hier schon nichts mehr zu finden ist, dann abbrechen
                if not df_merged.shape[0]:
                    bnix = True
                    break

                df_merged = df_merged.iloc[:,[0,1,2]].drop_duplicates() # spart hoffentlich Rechenleistung

                # am Ende neue Positionen zwischenspeichern
                if idx_group == len_groups:
                    #print(span_values,"am Ende neue Positionen zwischenspeichern")
                    self.positions = [(snr, std, end) for snr, std, end in df_merged.values]
                    
                #[display(group.loc[group["SatzNr"].isin(check_ids)]),display(df_merged.loc[df_merged["SatzNr"].isin(check_ids)])]

                # ggf. Such-Gruppen, die zusammengehören sollen, filtern

        # Auswahl bereinigen und sortieren (list type auch wichtig für das Indexieren mit Pandas)
        # falls Vagheitssätze drin sein dürfen
        if not bnix:
            snrn_set = {snr for snr, std, end in self.positions}
            if self.bvagheit:
                self.positions = sorted([(snr, std, end) for snr, std, end in set(self.positions)
                                     if snr in snrn_set])
            # falls ich Vagheitssätze ausschließen möchte
            else:
                self.positions = sorted([(snr, std, end) for snr, std, end in set(self.positions)
                                     if snr in snrn_set and snr not in self.vagheitsliste])

        if not self.positions:
            bnix = True
    
        # Prüfen, ob was gefunden wurde, wenn ja, Duplikate raus und sortieren
        if bnix:
            print("mit %s alles rausgefiltert"%span_values)
            
        else:
            self.positions = sorted(list(set(self.positions)), key=itemgetter(0,1))
            self.snrn = sorted(list(set(snr for snr, st, en in self.positions)))
            print("%i Sätze/Satz, %i Position/en"%(len(self.snrn),len(self.positions)))

            # Dateien zum schnellen Nachschlagen in Dictionary speichern
            if "Datei" in self.dfsaetze:
                self.dic_snrn_dateien = {snr:datei for snr, datei in self.dfsaetze.loc[:,[self.SNR,"Datei"]].values}
            
            # Ergebnis zur Überprüfung anzeigen
            #print("bla", datetime.datetime.fromtimestamp(time.time()))
            #print(baendern)


    # Anzeige-Funktionen ##################################################################
    
    # muss ich identity_cols und positions übergeben?
    def display_everything(self, df=None):
        with pd.option_context("display.max_rows",None,"display.max_colwidth",-1):
            display(df)
    
    def posis_tabelle(self, df=None):
        # Annotationen in Tabelle anzeigen lassen
        if df is None: # ein self.... kann kein Standard-Argument sein ...
            df = self.dfannos
        self.display_everything(df.set_index(self.identity_cols).loc[self.positions])
            
    def saetze_tabelle(self, df=None):
        # alle Sätze in einer Tabelle anzeigen lassen
        if df is None: # ein self.... kann kein Standard-Argument sein ...
            df = self.dfsaetze
        self.display_everything(df.loc[df[self.SNR].isin(self.snrn)])
    
    def posis_satz(self, snr, df=None):
        # alle Annotationen der gefilterten Sätze anzeigen lassen
        if df is None: # ein self.... kann kein Standard-Argument sein ...
            df = self.dfannos
        self.display_everything(df.loc[df[self.SNR]==snr])
            
    def filter_brat(self,pfad_brat_filter="",pfad_brat_quelle="",url="",izfill=4):
        #http://127.0.0.1:8001/index.xhtml#/Masterarbeit/Analyse/
        
        if len(self.snrn) > 1:
            # Dateien löschen, die vorher darin waren
            for datei in os.listdir(pfad_brat_filter):
                if datei.endswith(".txt") or datei.endswith(".ann"):
                    os.remove(os.path.join(pfad_brat_filter, datei))

            # Dateien hineinkopieren
            for snr in self.snrn:
                ssnr = str(snr).zfill(izfill)
                os.system("cp " + pfad_brat_quelle + ssnr + ".txt " + pfad_brat_filter + ssnr + ".txt")
                os.system("cp " + pfad_brat_quelle + ssnr + ".ann " + pfad_brat_filter + ssnr + ".ann")

            # die neue erste Datei anzeigen
            ssnr = str(self.snrn[0]).zfill(izfill)
            #with open("zeug.json", "w") as zeug:
            #    json.dump({"Bratdatei": ssnr}, zeug)
            display(Markdown("["+ssnr+"]("+url+ssnr+")"))
        else:
            print("Wohl nichts gefunden")
            
    def brat_gesamtdok(self):
        
        # Filtern und Strings in Zusatz in Python-Objekte umwandeln
        self.dfsaetze = self.dfsaetze.loc[self.dfsaetze[self.SNR].isin(self.snrn)]
        self.dfannos = self.dfannos.loc[self.dfannos[self.SNR].isin(self.dfsaetze[self.SNR].values)]
        self.dfannos["Zusatz"] = self.dfannos.loc[self.dfannos["Zusatz"].notna(),"Zusatz"].\
                                                apply(lambda x: literal_eval(x))
        
        # für neues Dokument Anfangs- und Endpositionen der Sätze und Annotationen neu berechnen

        # Positionen der Sätze neu berechnen
        self.dfsaetze["Satzlänge"] = self.dfsaetze.loc[:,"Satz"].apply(lambda x: len(x.strip()))
        self.dfsaetze["Ende"] = self.dfsaetze.loc[:,"Satzlänge"].cumsum()
        self.dfsaetze.loc[:,"Ende"] += [i for i in range(self.dfsaetze.shape[0])]
        self.dfsaetze["Start"] = self.dfsaetze["Ende"].sub(self.dfsaetze["Satzlänge"].values)

        # Anzahl der Annotationen pro Satz feststellen
        snr_counted = Counter(self.dfannos[self.SNR].values)

        # für die Annotationen die Token-Nummern erstellen - erstmal überflüssig
        #self.dfannos["Tnr"] = [tnr for snr in snr_counted for tnr in range(1,snr_counted[snr]+1)]

        # Positionen der Token neu berechnen
        # im Moment keine Sätze mit Fragmenten zum Testen des Codes
        # ich könnte auch len() und cumsum() noch mal benutzen, für Zusatz dann hier schon dspans
        sub_satz_starts = [

            start_annosatz - start_satz

            # Startposition des ersten Tokens und des neuberechneten Satzbeginns pro Satz
            for (snr, start_annosatz), start_satz

                in zip(
                    # nach Satz-Nummer wird gruppiert, die Werte brauche ich trotzdem, deshalb assign
                    self.dfannos.assign(Snr = self.dfannos[self.SNR].values).\
                    # durch Gruppierung jeden ursprünglichen Satzbeginn bekommen
                    groupby(self.SNR).first().loc[:,["Snr","StartDok"]].values

                    ,
                    self.dfsaetze["Start"].values
                )

            # in der Anzahl der Token pro Satz für Übereinstimmung mit self.dfannos.shape[0]
            for _ in range(snr_counted[snr])
        ]

        # alle Positionen neu berechnen
        self.dfannos["StartDok"] = self.dfannos.loc[:,"StartDok"].sub(sub_satz_starts)
        self.dfannos["EndeDok"] = self.dfannos.loc[:,"EndeDok"].sub(sub_satz_starts)
        self.dfannos["Zusatz"] = [

            [se - sst for se in zus]
            if isinstance(zus, list) and isinstance(zus[0], int) else
            [[[st - sst, en - sst] for st, en, in zus[0]], zus[1]]
            if isinstance(zus, list) else
            (zus[0] - sst, zus[1] - sst, zus[2])
            if isinstance(zus, tuple) else
            None

            for zus, sst in zip(self.dfannos["Zusatz"].values, sub_satz_starts)
        ]
        
        # Arrays für die ann-Datei erstellen #################

        # dictionary für die Fragmente machen 
        # -> Sätze irgendwie noch mal aus dem DFKI holen, denn ...
        # ... hier ist bei den Fragmenten einfach kein Zusatz gespeichert ...
        
        # zum Zuordnen der IDs zu den Spans
        dspans = {
            (snr, std, end)
            if zus is None else
            (snr, zus[0], zus[1]):idx

            for idx, (snr, std, end, zus)
            in enumerate(self.dfannos.loc[
                self.dfannos["Typ"].isin(["ent","pos"]), [self.SNR,"StartDok","EndeDok","Zusatz"]
            ].values, start=1)
        }

        # Span-Annotationen eintragen -> Entitäten und POS-Annotationen
        ann_span = [

            # z.B. ['T1\tAPPR 0 5\tUnter\n'],
            "T%i\t"%nr
             + "%s %i %i\t"%(wert.replace("$(","_s").replace("$.","_p").replace("$,","_k"), std, end)
             + text + "\n"
            if zus is None else
            "T%i\t"%nr
             + "%s %i %i;%i %i\t"%(wert.replace("$(","_s").replace("$.","_p").replace("$,","_k"),
                             zus[0], zus[1], std, end)
             + text + "\n"
            if std != zus[0] else
            ""

            for nr, (snr, wert, text, std, end, zus)
             in enumerate(self.dfannos.loc[
                 self.dfannos["Typ"].isin(["pos","ent"]), [self.SNR,"Wert","Text","StartDok","EndeDok","Zusatz"]
             ].values, start=1)
        ]

        # relative Annotationen eintragen -> Relationen und Dependenz-Annotationen
        ann_rel = [

            # z.B. ['R1\tcase arg1:T1 arg2:T8\n'],
            "R%i\t"%nr + "%s arg1:T%s arg2:T%i"%(rel,dspans[(snr,st1,en1)],dspans[(snr,st2,en2)]) + "\n"
            for nr, (snr, rel, st1, en1, (st2, en2, text))

            # (self.dfannos["Zusatz"].notna()) -> root hat zwar dep, aber NaN als Zusatz
            in enumerate(self.dfannos.loc[
                (self.dfannos["Typ"].isin(["dep","rel"]))&(self.dfannos["Zusatz"].notna()),
                [self.SNR, "Wert", "StartDok", "EndeDok", "Zusatz"]
            ].values, start=1)
        ]

        datts = {"past":"DocTime","past_present":"DocTime","future":"DocTime","present":"DocTime",
                 "negative":"LevelOfTruth","possible_future":"LevelOfTruth","positive":"LevelOfTruth",
                 "speculated":"LevelOfTruth","unlikely":"LevelOfTruth",
                 "patient":"Person","family":"Person","doctor":"Person","other":"Person","Patient":"Person"}

        # Attribute eintragen
        ann_att = [

            # z.B. ['A1\tLevelOfTruth T12 speculated\n'],
            "A%i\t"%nr + "%s T%i %s\n"%(datts[wert], dspans[snr,std,end], wert)
            if zus is None or std == zus[0] else
            ""

            for nr, (snr, wert, std, end, zus)
             in enumerate(self.dfannos.loc[
                 self.dfannos["Typ"]=="att", [self.SNR,"Wert","StartDok","EndeDok", "Zusatz"]
             ].values, start=1)
        ]
        
        # Dateien erstellen, alte evtl. löschen ##############

        # alle Annotationen und Sätze zu jeweils einer einzigen Variable zusammenführen
        ann_datei = ann_span + ann_rel + ann_att
        text_datei = [satz.strip() + "\n" for satz in self.dfsaetze["Satz"].values]

        #""" Dateien löschen, die vorher darin waren
        if self.bdateien_loeschen:
            for datei in os.listdir(self.path_brat):
                if datei.endswith(".ann") or datei.endswith(".txt"):
                    os.remove(self.path_brat+datei) #"""

        def save_brat_file(name="",typ="",datei=""):
            with open(self.path_brat+name+typ, "w") as d:
                d.writelines(datei)
        
        # Annotations- und Satz-Dateien (.ann, .txt) anlegen
        save_brat_file(self.brat_ges_dok,".txt",text_datei)
        save_brat_file(self.brat_ges_dok,".ann",ann_datei)


## Zeug laden

In [40]:
# Falls ich Brat benutzen möchte
url = "http://127.0.0.1:8001/index.xhtml#/Masterarbeit/Analyse/"
pfad_brat_macss = "/mnt/e/brat-v1.3_Crunchy_Frog/data/Masterarbeit/"
pfad_filter = pfad_brat_macss + "Analyse/"
pfad_quelle = pfad_brat_macss + "Analyse-Brat/"

In [66]:
pfad_datenbank = "/mnt/e/Ganz_doll_wichtiges/DFKI/Masterarbeit/Datenbank/"
korpus_dateien = [["dfalles.csv", "dfsaetze.csv"],
                  ["Zusatzsätze/dfzuhause_annos_adverbien.csv", "Zusatzsätze/dfzuhause_saetze_adverbien.csv"],
                  ["Zusatzsätze/dfzuhause_annos_adverbien2.csv", "Zusatzsätze/dfzuhause_saetze_adverbien2.csv"],
                  ["Zusatzsätze/dfzuhause_annos_adjektive.csv", "Zusatzsätze/dfzuhause_saetze_adjektive.csv"]
                 ][0]

# Vorlagen zum Schnelleren Suchen speichern
suchnonf = ["future","possible_future","unlikely","speculated"] #alle nonfaktischen Attributwerte
suchnomen = ["NN","NE","FM"] #alle nominalen POS-Tags
suchfinverb = ["VVFIN","VMFIN","VAFIN"] #alle finiten Verben
suchinfverb = ["VVINF","VMINF","VAINF"] #alle infiniten Verben
suchpartverb = ["VVPP","VMPP","VAPP"] #alle Partizipien
suchverb = [verb for verbs in zip(suchfinverb,suchinfverb,suchpartverb) for verb in verbs] #alle verbalen POS-Tags
suchkonj = ["KOUI","KOUS","KON","KOKOM"] #alle Konjunktionen
suchintsatzz = ["$,","$("] #alle internen Satzzeichen
suchkoord = [koord for koords in zip(suchkonj,suchintsatzz) for koord in koords] #alle evtl. Koordinationen
suchinterrrela = ["PRELS","PRELAT","PWS","PWAT","PWAV"] #alle Relativpronomen und Interrogativpronomen
suchpraepos = ["APPR","APPRART","APPO"] #alle Präpositionen
suchmod = ["amod","nmod","advmod"]
vaguenesstrigger = "ca\.|ca |rel\.|relativ|recht |etwa|ungefaehr" #alle Vagheits-Auslöser

def exklusiv(suchwerte=None):
    return ["-" + wert for wert in suchwerte]

## Filtern

In [69]:
# Suche
# solang ich KomplSuchKorp bearbeite, muss das in dieser Zelle bleiben
KSK = KomplSuchKorp(pfad_datenbank+korpus_dateien[0], pfad_datenbank+korpus_dateien[1])

# contains statt regex, wenn regAusdr nicht so komplex und wenn es zu lange dauert
# contains (re) bekommt z.B. nicht hin: r"(?<!unwahrscheinlich.*)kein"
#KSK.bcontains = True

KSK.values = []
KSK.searchstring = r"wurde die wahrscheinlichste"

KSK.suche()

1 Sätze/Satz, 31 Position/en


In [70]:
# Sätze anzeigen
KSK.saetze_tabelle()

,SatzNr,Länge,Satz
639,4619,226,"Es wurde die wahrscheinlichste Diagnose einer Papillitis gestellt, aufgrund auffaelliger Marklagerlaesionen in einer cerebralen Kernspintomographie erfolgte zur weiteren Abklaerung eine Verlegung auf die neurologische Station."


In [5]:
# Annotationen anzeigen
""" Zusätzlicher Filter
dfannos_fil = KSK.dfannos.set_index(KSK.identity_cols).loc[KSK.positions]
dfannos_fil = dfannos_fil.loc[dfannos_fil["Typ"]=="att"]
KSK.display_everything(dfannos_fil) #"""
KSK.posis_tabelle()

Unnamed: 0  Datei  Typ              Text  \
SatzNr StartDok EndeDok                                             
4395   188      192      106406      12673  pos  Koro               
                192      106407      12673  dep  Koro               
                192      106408      12673  ent  Koro               
                192      106409      12673  rel  Koro               
                192      106410      12673  rel  Koro               
       193      196      106411      12673  pos  vor                
                196      106412      12673  dep  vor                
                209      106413      12673  ent  vor ca. 3 Jahren   
       197      200      106414      12673  pos  ca.                
                200      106415      12673  dep  ca.                
       201      202      106416      12673  pos  3                  
                202      106417      12673  dep  3                  
       203      209      106418      12673  pos  Jahren             
                209      106419      12673  dep  Jahren             
       210      214      106420      12673  pos  ohne               
                214      106421      12673  dep  ohne               
                214      106422      12673  ent  ohne               
       215      223      106423      12673  pos  Stenosen           
                223      106424      12673  dep  Stenosen           
                223      106425      12673  ent  Stenosen           
                223      106426      12673  att  Stenosen           
       224      232      106427      12673  pos  offenbar           
                232      106428      12673  dep  offenbar           
       232      233      106429      12673  pos  .                  
                233      106430      12673  dep  .                  

                                      Wert                              Zusatz  
SatzNr StartDok EndeDok                                                         
4395   188      192      NN                 NaN                                 
                192      root               NaN                                 
                192      Medical_condition  NaN                                 
                192      hasState           [[[215, 223]], 'Stenosen']          
                192      has_time_info      [[[193, 209]], 'vor ca. 3 Jahren']  
       193      196      APPR               NaN                                 
                196      case               (203, 209, 'Jahren')                
                209      Time_information   NaN                                 
       197      200      ADV                NaN                                 
                200      amod               (203, 209, 'Jahren')                
       201      202      CARD               NaN                                 
                202      nummod             (203, 209, 'Jahren')                
       203      209      NN                 NaN                                 
                209      obl                (224, 232, 'offenbar')              
       210      214      APPR               NaN                                 
                214      case               (215, 223, 'Stenosen')              
                214      Negation           NaN                                 
       215      223      NN                 NaN                                 
                223      obl                (224, 232, 'offenbar')              
                223      Medical_condition  NaN                                 
                223      negative           NaN                                 
       224      232      ADJD               NaN                                 
                232      amod               (188, 192, 'Koro')                  
       232      233      $.                 NaN                                 
                233      punct              (188, 192, 

In [11]:
# Annos eines einzelnen Satzes anzeigen
""" Zusätzlicher Filter
dfannos_fil = KSK.dfannos.loc[(KSK.dfannos["Typ"]=="dep")
                              &(KSK.dfannos[KSK.SNR]==15)
                             ]
KSK.display_everything(dfannos_fil) #"""
KSK.posis_satz(4395)

,Unnamed: 0,Datei,SatzNr,StartDok,EndeDok,Typ,Text,Wert,Zusatz
504,106406,12673,4395,0,4,pos,Koro,NN,None
505,106407,12673,4395,0,4,dep,Koro,root,None
506,106408,12673,4395,0,4,ent,Koro,Medical_condition,None
507,106409,12673,4395,0,4,rel,Koro,hasState,"[[[27, 35]], Stenosen]"
508,106410,12673,4395,0,4,rel,Koro,has_time_info,"[[[5, 21]], vor ca. 3 Jahren]"
509,106411,12673,4395,5,8,pos,vor,APPR,None
510,106412,12673,4395,5,8,dep,vor,case,"(15, 21, Jahren)"
511,106413,12673,4395,5,21,ent,vor ca. 3 Jahren,Time_information,None
512,106414,12673,4395,9,12,pos,ca.,ADV,None
513,106415,12673,4395,9,12,dep,ca.,amod,"(15, 21, Jahren)"


In [9]:
# Sätze einzeln in Brat-Ordner aus Quellordner kopieren
KSK.filter_brat(pfad_brat_filter=pfad_filter,pfad_brat_quelle=pfad_quelle,url=url)

Wohl nichts gefunden


In [10]:
# Sätze in einer Brat-Datei sammeln
#KSK.brat_ges_dok = "test"
KSK.brat_gesamtdok()

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
# gespeicherte Suchen
#["if_consequence","consequence"],exklusiv(suchfinverb), exklusiv(suchpartverb)
#r"offensichtlich\b|offenbar\b|wahrscheinlich\b|vermutlich\b|mutmasslich\b"
# ["VVINF"],exklusiv(suchfinverb), exklusiv(suchpartverb)
# mellitus|fugax|spinalis|descendens|abdominalis|inferior|superior|carotis|subclavia|epilepticus

## zusätzliche Filterung

In [15]:
# ist auf die Zellen drüber angewiesen wegen dfmacss
from ast import literal_eval
print("los")
dfmacss_fil2 = KSK.dfannos.set_index(KSK.identity_cols).loc[KSK.positions].reset_index()
dfmacss_fil2["Zusatz"] = dfmacss_fil2.Zusatz.apply(lambda cell: literal_eval(cell) if cell != "" else None)
dfmacss_fil2_mod_nonf = dfmacss_fil2.loc[dfmacss_fil2["Wert"].isin(suchmod+suchnomen)]

# Dependenzen von Attributen durch Gruppierung trennen
dfmacss_fil2_grouped = dfmacss_fil2_mod_nonf.groupby("Typ", sort=False)
groups = [dfmacss_fil2_grouped.get_group(key).loc[:,identity_cols+["Typ","Zusatz"]]
          for key in dfmacss_fil2_grouped.groups]

# feststellen, welche Gruppe welche Gruppe ist
if groups[0].iloc[(0,3)] == "dep":
    idep = 0
    iatt = 1
else:
    iatt = 0
    idep = 1

# Positionen des syntaktischen Skopus extrahieren
refs_dep_posis = [(snr,)+zus[:2] for snr,zus in groups[idep].loc[:,["SatzNr","Zusatz"]].values]

# syntaktischen Skopus herausfiltern, der nicht eines der Attribute hat
att_posis_set = {(snr,st,en) for snr,st,en in groups[iatt].loc[:,["SatzNr","StartDok","EndeDok"]].values}
refs_dep_filt = [posis if posis in att_posis_set else None for posis in refs_dep_posis]

# Modifikatoren (amod, nmod, advmod) herausfiltern, die hinter ihrem syntaktischen Skopus liegen
groups[idep]["Refs"] = refs_dep_filt
dfdep = groups[idep].copy() # kann das mit unterer Zeile kombiniert werden?
dfdep = dfdep.loc[dfdep["Refs"].notna()]
dfdep["RefEnd"] = [ref[2] for ref in refs_dep_filt if ref is not None] # hier dfdep["Refs"] ohne if und slicen verwenden?
dfdep_att = dfdep.query("RefEnd < EndeDok")

# DataFrame mit den Ergebnissen erstellen
positions_dep_att = [tuple(idx) for idx in dfdep_att.loc[:,["SatzNr","StartDok","EndeDok"]].values]
#snrn_dep_att = dfdep_att["SatzNr"].values
#dfmacss.loc[dfmacss["SatzNr"].isin(snrn_dep_att)]
dfmacss_dep_att = dfmacss.set_index(identity_cols).loc[positions_dep_att].reset_index()
print("fertig")
#with pd.option_context("display.max_rows",None):
#    display(dfmacss_dep_att.loc[dfmacss_dep_att["Typ"]=="dep"])

los


KeyError: "None of [MultiIndex(levels=[[4395], [188, 193, 197, 201, 203, 210, 215, 224, 232], [192, 196, 200, 202, 209, 214, 223, 232, 233]],\n           codes=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 4, 2, 3, 4, 5, 6, 7, 8]],\n           names=['SatzNr', 'StartDok', 'EndeDok'])] are in the [index]"

In [41]:
# dictionary erstellen mit Satznummer: Skopus/Skopi: entsprechende Modifikatoren
print("los")
ddep_atts = {}

"""
dfmacss_da_grouped = dfmacss_dep_att.loc[dfmacss_dep_att["Typ"]=="dep"].groupby("SatzNr", sort=False)
dfgroups = [dfmacss_da_grouped.get_group(key) for key in dfmacss_da_grouped.groups]
print("for")
for idx, snr in enumerate(set(dfmacss_dep_att["SatzNr"].values)):
    dffil = dfgroups[idx] # """

for snr in set(dfmacss_dep_att["SatzNr"].values):
    dffil = dfmacss_dep_att.loc[(dfmacss_dep_att["SatzNr"]==snr)&(dfmacss_dep_att["Typ"]=="dep")]
    ddep_atts[snr] = {}
    for zus in dffil["Zusatz"].unique():
        ddep_atts[snr][zus] = []
        for t,s,e in dffil.loc[dffil["Zusatz"]==zus,["Text","StartDok","EndeDok"]].values:
            ddep_atts[snr][zus].append([t,[s,e]])

# Sätze evtl. weiter filtern: Sätze mit mehr als einem Modifikator pro Skopus
dep_atts = [[snr,dep,zus] for snr,v in ddep_atts.items() for zus,dep in v.items() if len(dep) > 1]
snrn_atts_mehrere_deps = sorted([d[0] for d in dep_atts])
display(len(dep_atts))
#dep_atts

los


105

### Brat-Dateien erstellen und Link dazu anzeigen lassen

In [19]:
# Brat-Dateien erstellen und Link dazu anzeigen lassen
pfadbrat = "/var/www/html/brat-v1.3_Crunchy_Frog/data/MACSS/temp_korpus/"
print("los")
dfmacss = KSK.dfannos
#snrn_atts_mehrere_deps
dfmacss_brat = dfmacss_brat.loc[dfmacss["SatzNr"].isin(KSK.snrn)].\
                        sort_values(["SatzNr","StartDok"]).copy()
dfmacss_brat["Zusatz"] = dfmacss_brat.Zusatz.apply(lambda r: literal_eval(r) if r != None else "")

# Positionen neu berechnen ############################################################################## 

# ein dict für die Start- und End-Positionen pro Satz
dstarts_sub = {
    snr:start for snr, start
    in dfmacss_brat.groupby("SatzNr").first()["StartDok"].items()
}

dfmacss_brat["StartDok"] = [st-dstarts_sub[snr] for st,snr in dfmacss_brat.loc[:,["StartDok","SatzNr"]].values]
dfmacss_brat["EndeDok"] = [en-dstarts_sub[snr] for en,snr in dfmacss_brat.loc[:,["EndeDok","SatzNr"]].values]
dfmacss_brat["Zusatz"] = [
    (zus[0]-dstarts_sub[snr], zus[1]-dstarts_sub[snr], zus[2])
    if isinstance(zus, tuple) else
    [[[z-dstarts_sub[snr] for z in zu] for zu in zus[0]],zus[1]]
    if isinstance(zus, list) and isinstance(zus[0], list) else
    [zus[0]-dstarts_sub[snr], zus[1]-dstarts_sub[snr]]
    if isinstance(zus, list) and isinstance(zus[0], int) else
    ""
    for zus, snr in dfmacss_brat.loc[:,["Zusatz","SatzNr"]].values
]
#"""
dep_atts_sub = [[snr, [[text]+[st-dstarts_sub[snr],en-dstarts_sub[snr]] for text,(st,en) in tokens], zusatz]
            for [snr, tokens, zusatz] in dep_atts if snr in dstarts_sub] # """

# Spans (Entitäten, POS) ################################################################################ 

# Zusatz-Positionen der Fragmente in ein dict
dspans_fragm = {}
dspans_fragm_erste = []
for snr,st,en,zus in dfmacss_brat.loc[(dfmacss_brat["Typ"]=="ent")
                                      &(dfmacss_brat["Zusatz"]!=""),
                                      ["SatzNr","StartDok","EndeDok","Zusatz"]
                                     ].values:
    
    if (snr,st,en) not in dspans_fragm:
        dspans_fragm[(snr,st,en)] = []
    
    # falls 1. Fragment Positionen des 2. Fragments im Zusatz enthält statt andersrum
    if (snr,zus[0],zus[1]) not in dspans_fragm and zus[0] != st:
        dspans_fragm[(snr,st,en)].append([zus[0],zus[1]])
        dspans_fragm[(snr,zus[0],zus[1])] = [[st,en]]
        dspans_fragm_erste.append((snr,st,en))
        
    # 2. Fragment
    else:
        if zus[0] != st:
            dspans_fragm_erste.append((snr,zus[0],zus[1]))
            dspans_fragm[(snr,st,en)].append([zus[0],zus[1]])
            dspans_fragm[(snr,zus[0],zus[1])].append([st,en])
        
dspans_fragm_erste = set(dspans_fragm_erste)

# ein dict für die Span-Nummern die Anzahl der Token pro Satz
dpos_ents_count = {
    snr:count for snr,count
    in dfmacss_brat.loc[dfmacss_brat["Typ"].isin(["pos","ent"])].groupby("SatzNr").count()["Datei"].items()
}

# durchgehende Nummerierung pro Satz
spannummern = [nr for count in dpos_ents_count.values() for nr in range(1,count + 1)]

#""" Span-Annotationen eintragen -> Entitäten und POS-Annotationen
ann_span = [
    
    [snr, "T%i"%nr,
     "%s %i %i"%(wert.replace("$(","_s").replace("$.","_p").replace("$,","_k"), std, end),
     text]
    if zus == "" else
    [snr, "T%i"%nr,
     "%s "%(wert.replace("$(","_s").replace("$.","_p").replace("$,","_k")) + "%i %i;"%(std, end)
     + ";".join(["%i %i"%(st, en) for st, en in dspans_fragm[(snr,std,end)]]),
     text]
    if (snr,std,end) in dspans_fragm_erste else
    "" # der zweite Teil eines Fragments -> unbrauchbar, lässt sich schwer vorher rausfiltern
    
    for (snr, wert, text, std, end, zus), nr
     in zip(dfmacss_brat.loc[
         dfmacss_brat["Typ"].isin(["pos","ent"]), ["SatzNr","Wert","Text","StartDok","EndeDok","Zusatz"]
         ].values
         , spannummern
           )
     
] #"""

ann_span = [a for a in ann_span if a != ""]

#"""
ann_span += [
    [snr, "T%i"%(nr1+st), "S %i %i"%(st, en), text]
    for nr1, (snr, token, zus) in enumerate(dep_atts_sub, start=10000)
     for text, st, en in token
] # """

# Vorbereitung ##########################################################################################

# Index für die Spans für die Zuweisung bei den Relationen und Attributen
dspans = {
    (snr, std, end, typ)
    #if zus is "" else
    #(snr, zus[0], zus[1], typ)
    :nr
    
    for (snr, std, end, zus, typ), nr
    in zip(
        dfmacss_brat.loc[
            dfmacss_brat["Typ"].isin(["ent","pos"]), ["SatzNr","StartDok","EndeDok","Zusatz","Typ"]
            ].values
        , spannummern
        )
}

# Attribute #############################################################################################

datts = {"past":"DocTime","past_present":"DocTime","future":"DocTime","present":"DocTime",
         "negative":"LevelOfTruth","possible_future":"LevelOfTruth","positive":"LevelOfTruth",
         "speculated":"LevelOfTruth","unlikely":"LevelOfTruth",
         "patient":"Person","family":"Person","doctor":"Person","other":"Person","Patient":"Person"}

# Attribute eintragen
ann_att = [
    
    [snr, "A%i"%nr, "%s T%i %s"%(datts[wert], dspans[(snr,std,end,"ent")], wert), ""]
    if zus == "" or std == zus[0] else
    ""
    
    for nr, (snr, wert, std, end, zus, typ)
     in enumerate(dfmacss_brat.loc[
         dfmacss_brat["Typ"]=="att", ["SatzNr","Wert","StartDok","EndeDok", "Zusatz", "Typ"]
     ].values, start=1)
    if wert in datts # um alte Sachen wie conditional auszuschließen
]

ann_att = [a for a in ann_att if a != ""]

# Relationen (Relationen, Dependenzen) ##################################################################

dfmacss_brat_rel = dfmacss_brat.loc[(dfmacss_brat["Typ"].isin(["dep","rel"]))&(dfmacss_brat["Zusatz"]!="")].copy()

# Zusatz normalisieren
dfmacss_brat_rel["Zusatz"] = [(zus[0],zus[1]) if isinstance(zus, tuple) else
                          (zus[0][0][0],zus[0][0][1]) if isinstance(zus, list) else
                          "" for zus in dfmacss_brat_rel["Zusatz"].values]

# ein dict für die durchgehende Nummerierung die Anzahl der Relationen pro Satz
ddep_rels_count = {
    snr:count for snr,count
    in dfmacss_brat.loc[
        (dfmacss_brat["Typ"].isin(["dep","rel"]))
        &(dfmacss_brat["Wert"]!="root")
    ].groupby("SatzNr").count()["Datei"].items()
}

# durchgehende Nummerierung pro Satz
relnummern = [nr for count in ddep_rels_count.values() for nr in range(1,count + 1)]

# ich könnte die Relationen noch korrigieren, damit sie bei Fragmenten nicht die falschen Spans enthalten
# Beispiel Satz 3444 -> korrekter Span wäre 3 5
    #82153 	12477 	3444 	29 	36 	rel 	RR systol. 	has_measure 	[[[37, 62]], 'zwischen 130 und 145 mmHg']
    #3 	5 	ent 	RR systol. 	Biological_parameter 	[29, 36]
# Beispiel Satz 11441 -> korrekter span wäre 96, 121
    #303742 	51 	11441 	96 	121 	ent 	antidepressive Behandlung empfohlen 	Treatment 	[96, 121]
    #303796 	51 	11441 	199 	208 	ent 	antidepressive Behandlung empfohlen 	Treatment 	[96, 121]
    #303797 	51 	11441 	199 	208 	rel 	antidepressive Behandlung empfohlen 	involves 	[[[126, 136], [139, 147]], Agamaladin Valdoxan]
    #303798 	51 	11441 	199 	208 	rel 	antidepressive Behandlung empfohlen 	involves 	[[[172, 184]], Escitalopram]
    
# Relationen eintragen
ann_rel = [
    
    [snr, "R%i"%nr, "%s arg1:T%s arg2:T%i"%(rel,dspans[(snr,st1,en1,"ent")],dspans[(snr,st2,en2,"ent")]), ""]
    if typ == "rel" and (snr,st1,en1,"ent") in dspans and (snr,st2,en2,"ent") in dspans else
    [snr, "R%i"%nr, "%s arg1:T%s arg2:T%i"%(rel,dspans[(snr,st1,en1,"pos")],dspans[(snr,st2,en2,"pos")]), ""]
    if (snr,st1,en1,"pos") in dspans and (snr,st2,en2,"pos") in dspans else
    ""
                                                            
    for (snr, rel, st1, en1, (st2, en2), typ), nr
    
    # (ann_rel["Zusatz"].notna()) -> root hat zwar dep, aber NaN als Zusatz
    in zip(
        dfmacss_brat_rel.loc[(dfmacss_brat_rel["Typ"].isin(["dep","rel"]))&(dfmacss_brat_rel["Zusatz"]!=""),
                             ["SatzNr", "Wert", "StartDok", "EndeDok", "Zusatz", "Typ"]].values
        , relnummern
    )
    #if ((snr,st1,en1,"ent") in dspans or (snr,st1,en1,"pos") in dspans)
    #and ((snr,st2,en2,"ent") in dspans or (snr,st2,en2,"pos") in dspans)
    # 2226 # ein nicht korrekt gesplitteter Satz, um den ich mich nicht kümmern möchte
    # 2669 # aus irgendeinem Grund in der rel flasche Positionen des Fragments
    # 599  # is_located über die Satzgrenze hinweg
    # 6348 # consequence über die Satzgrenze hinweg
    # 763  # Relation über die Satzgrenze hinweg
    # 671  # falsch gesplitteter Satz
    # 668  # falsch gesplitteter Satz
    # 6985 # falsch gesplitteter Satz
    # 350  # falsch gesplitteter Satz
    # 11336 # examines über die Satzgrenze hinweg
    # 11575 # aus irgendeinem Grund in der rel flasche Positionen des Fragments
    # 11675 # aus irgendeinem Grund in der rel flasche Positionen des Fragments
]

ann_rel = [a for a in ann_rel if a != ""]

# Annotationen speichern ################################################################################

# DataFrame mit allen Annotationen erstellen und Gruppieren
dfbrat = pd.DataFrame(sorted(ann_span + ann_att + ann_rel, key=itemgetter(0)),
                      columns=["SatzNr","ID", "Wert","Zusatz"])
dfbrat_grouped = dfbrat.groupby("SatzNr", sort=False)
brat_groups = [dfbrat_grouped.get_group(key) for key in dfbrat_grouped.groups] #+["Typ"]

#""" Dateien löschen, die vorher darin waren
print("alte Dateien löschen")
for datei in os.listdir(pfadbrat):
    if not "conf" in datei:
        os.remove(pfadbrat+datei)
#"""

# pro Satz eine Datei im Brat-Ordner speichern # snrn_atts_mehrere_deps
print(len(snrn)*2, "Dateien speichern")
for brat_group, satz in zip(brat_groups,
                            dfmsaetze.loc[dfmsaetze["SatzNr"].isin(snrn),"Satz"].values):
    snr = str(brat_group.iloc[0,0])
    with open(pfadbrat + snr + ".txt", "w") as d:
        d.write(satz.strip())
    brat_group.loc[:,["ID", "Wert","Zusatz"]].to_csv(pfadbrat + snr + ".ann", sep="\t", index=False, header=False)
print("fertig")

los


ValueError: malformed node or string: [[[27, 35]], 'Stenosen']

640 Dateien -> letzte angeschaute Datei: 351 (suchmod vor oder nach dem suchnonf)

In [51]:
# Anzeigen lassen #######################################################################################

snr = str(snrn[0])
display(Markdown("["+snr+"](http://localhost/brat-v1.3_Crunchy_Frog/#/MACSS/temp_korpus/"+snr+")"))

[6](http://localhost/brat-v1.3_Crunchy_Frog/#/MACSS/temp_korpus/6)

# 1-2-Tabellen-Korpus
<a href="#Dieses-Notebook">hoch</a>

In [2]:
# Dateien laden
pfadoliver = "/home/olli/Documents/MACCS_Masterarbeit/oliver/"

# Daten der manuellen Annotation
dfcom = pd.read_csv(pfadoliver+"brat_coms.csv",sep="\t")
dfrel = pd.read_csv(pfadoliver+"brat_rels.csv",sep="\t")
dfent = pd.read_csv(pfadoliver+"brat_ents.csv",sep="\t")
dfatt = pd.read_csv(pfadoliver+"brat_atts.csv",sep="\t")
with open("/var/www/html/pythonstuff/vagheitsaetze.tsv", "r") as f:
    vagheitssaetze = f.read().split("||")
    
# Daten der automatischen Annotation
dfflairposdepposi = pd.read_csv(pfadoliver+"dfflairposdepposi.csv",sep="\t",index_col=0)
dfsaetze = pd.read_csv(pfadoliver+"sentences.csv",sep="\t")

In [177]:
# Starts und -Enden zusammentragen =======================================================================

identity_cols = ["SatzNr","StartDok","EndeDok",]

# beim Erstellen der Indizes: Brief_0017_brat -> 17 if "B..." else 12371_anymous_brat -> 12371


"""# Entitäten-Indizes: benutze ich mit dem aktuellen Aufbau (ein DataFrame für alles) nicht
# [11884, 0, 0, 8, 0, 'Aufnahme'], 
# [Datei-Nr, SatzNr, StartDok, EndeDok, Index Gesamt, String],
indices_ent = [
    
    [int(docid[6:10]), sid, stdoc, endoc, ind, symbols]
    if docid[0] == "B" else
    [int(docid[:5]), sid, stdoc, endoc, ind, symbols]
    
    for (sid, stdoc, endoc, docid, symbols), ind
    in zip(dfent.loc[:,identity_cols+["Datei","String"]].values,
           dfent.index)
]#"""

# Fragmente-Indizes: 
# [11886, 7, 246, 251, 21, 'RR syst.'],
# [Datei-Nr, SatzNr, StartDok, EndeDok, Index Gesamt 1. Fragment, String],
# die Fragemente können dem jeweils ersten Fragment über den Gesamtindex einfach zugewiesen werden
dfent_fragm = dfent.loc[dfent["FragmPosi"].notna()]

indices_fragm = [
    
    [int(docid[6:10]), snr, frst, fren, ind, sstr]
     if docid[0] == "B" else
    [int(docid[:5]), snr, frst, fren, ind, sstr]
    
     for (docid, snr, frs, sstr), ind
     in zip(dfent_fragm.loc[:,["Datei","SatzNr","FragmPosi","String"]].values,
           dfent_fragm.index)
     
     # [[5734, 5744]] -> (5734, 5744); [[4189, 4192], [4233, 4236]] -> (4189, 4192) (4233, 4236)
     # die strings in Spalte FragmPosi werden mit ast.literal_eval() in lists umgewandelt
     for (frst, fren) in ast.literal_eval(frs)
]

"""# Token-Indizes: benutze ich mit dem aktuellen Aufbau (ein DataFrame für alles) nicht
# [11884, 0, 0, 8, 1, 'Aufnahme'],
# [Datei-Nr, SatzNr, StartDok, EndeDok, TokenIndex pro Satz, String],
indices_tok = [
    
    [int(docid[6:10]), sid, stdoc, endoc, spanidsent, symbols]
    if docid[0] == "B" else
    [int(docid[:5]), sid, stdoc, endoc, spanidsent, symbols]
    
    for sid, stdoc, endoc, docid, spanidsent, symbols
    in dfflairposdepposi.loc[:,identity_cols+["Datei","Nr","Lemma"]].values
]
#"""

# DataFrame mit allem erstellen ============================================================================
# index, docID, satzID, start, end, symbols, annotType, type, additional
# ('pos', 'ADJA', None),
# ('dep', 'amod', (5,7)),

triple_ents = [
    
    # wenn es zusätzliche Fragmente gibt, zeigt der Datensatz immer auf das erste Fragment
    (snr, std, end, "ent", symbols, ent, (std, end)) 
    if fragmposi is not np.nan else
    (snr, std, end, "ent", symbols, ent, None) 
    
    for snr, std, end, symbols, ent, fragmposi
    in dfent.loc[:,identity_cols+["String","Entität","FragmPosi"]].values]

dic_entid_ges = {entid_ges:(std, end, ent) for entid_ges, (std, end, ent)
                in zip(dfent_fragm.index, dfent_fragm.loc[:,["StartDok","EndeDok","Entität"]].values)}

triple_fragm_ents = [
    (snr, std, end, "ent", symbols, dic_entid_ges[index_ges][2], dic_entid_ges[index_ges][:2])
    for docid, snr, std, end, index_ges, symbols
    in indices_fragm
]

triple_rels = [
    (snr, std, int(id1[id1.find(",")+2:id1.find("]")]), "rel", symbols, rel, 
     (int(id2[id2.find("[")+2:id2.find(",")]), 
      int(id2[id2.find(",")+2:id2.find("]")])
     )) 
    
    for snr, std, end, id1, symbols, rel, id2 in dfrel.loc[:,identity_cols+["ID","Entität","Relation","ID2"]].values
]

dic_strings = {idx: symbols for idx, symbols in dfent.loc[:,["ID","String"]].values}

triple_atts = [
    (snr, std, end, "att", dic_strings[idx], att, None)
    for snr, std, end, att, idx in dfatt.loc[:,identity_cols+["Attribut","ID"]].values
]

""" die lass ich erstmal außen vor, weil die nicht so interessant sind für die Analyse, siehe nächste cell
triple_coms = [
    (snr, std, end, "com", dic_strings[idx], com, None)
    for snr, std, end, com, idx in dfcom.loc[:,identity_cols+["Kommentar","ID"]].values
]"""

triple_poss = [
    (snr, std, end, "pos", symbols, pos, None)
    for snr, std, end, symbols, pos in dfflairposdepposi.loc[:,identity_cols+["Lemma","POS"]].values
]

dic_token_nr = {(snr,tnr):(std,end) for snr, std, end, tnr
               in dfflairposdepposi.loc[:,identity_cols+["Nr"]].values}

triple_deps = [

    (snr, std, end, "dep", symbols, dep, dic_token_nr[(snr,ref)])
    if ref != 0 else
    (snr, std, end, "dep", symbols, dep, (0,))    

    for snr, std, end, symbols, ref, dep in dfflairposdepposi.loc[:,identity_cols+["Lemma","Ref","Dep"]].values
]

# DataFrame mit allem erstellen
dfalles = pd.DataFrame(
    triple_ents + triple_fragm_ents + triple_rels + triple_atts + triple_poss + triple_deps,
    columns=["SatzNr","StartDok","EndeDok","Typ","String","Wert","Zusatz"]
)

# vllt. mal 2 DataFrames (relationell) ausprobieren ========================================================
# DataFrame erstellen: index, docID, satzID, start, end, content


# DataFrame erstellen: index, span_index, annotType, type, additional


# alles speichern ==========================================================================================
dfalles.to_csv("MACCS_Masterarbeit/oliver/dfalles.csv", sep="\t")

dfalles

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
0,0,0,8,ent,Aufnahme,Treatment,None
1,0,9,11,ent,TX,Treatment,None
2,1,0,10,ent,Entlassung,Treatment,None
3,1,11,13,ent,TX,Treatment,None
4,2,0,15,ent,Geht soweit gut,State_of_health,None
5,2,24,34,ent,allgemeine,Measurement,None
6,2,35,46,ent,Schlappheit,Medical_condition,None
7,3,48,62,ent,Vor zwei Tagen,Time_information,None
8,3,92,98,ent,Faeden,Medical_device,None
9,3,99,107,ent,entfernt,Treatment,None


In [10]:
# Kommentare sind nicht so interessant - hier der Beweis dafür
dfcom.loc[(~dfcom["Kommentar"].str.startswith("String-Variations"))&
          (~dfcom["Kommentar"].str.startswith("CUI:"))&
         (~dfcom["Kommentar"].str.startswith("Rechtschreib"))]

,SatzNr,Datei,StartDok,EndeDok,ID,Comment,Kommentar
89,93,11897_anymous_brat,191,201,"11897_anymous_brat[[191, 201]]Medical_condition",comment,Fehler: Beschwerden
554,493,11961_anymous_brat,37,41,"11961_anymous_brat[[37, 41]]Kommentar",comment,Unvollständige Deidentifizierung?
2639,2077,12523_anymous_brat,122,123,"12523_anymous_brat[[122, 123]]Kommentar",comment,Tippfehler
3172,2464,Brief_0010_brat,2686,2706,"Brief_0010_brat[[2686, 2706]]Measurement",comment,"wahrscheinlich Satzzeichenfehler: vergroessert,"
4141,3041,Brief_0016_brat,6089,6101,"Brief_0016_brat[[6089, 6101]]Treatment",comment,evtl. Rechtschreibfehler: Ueberernaehrung
4773,3410,Brief_0021_brat,3031,3034,"Brief_0021_brat[[3031, 3034]]Kommentar",comment,womöglich fehlt hier ein Datum
4792,3417,Brief_0021_brat,4025,4027,"Brief_0021_brat[[4025, 4027]]Kommentar",comment,Wahrscheinlich Rechtschreibfehler
4936,3493,Brief_0022_brat,6313,6331,"Brief_0022_brat[[6313, 6331]]Medical_condition",comment,"tring-Variations[stricture of anus (N), strict..."
5196,3645,Brief_0024_brat,3585,3595,"Brief_0024_brat[[3585, 3595]]Measurement",comment,Fehler: multifokal
6090,4190,Brief_0031_brat,3082,3086,"Brief_0031_brat[[3082, 3086]]Biological_chemistry",comment,wahrscheinlich Rechtschleibfehler: m-GT


## Dependenz-Informationen erweitern

In [65]:
dfalles = pd.read_csv("MACCS_Masterarbeit/oliver/dfalles.csv", sep="\t", index_col=0)
dfalles

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
0,0,0,8,ent,Aufnahme,Treatment,NaN
1,0,0,8,pos,Aufnahme,NN,NaN
2,0,0,8,dep,Aufnahme,root,NaN
3,0,9,11,ent,TX,Treatment,NaN
4,0,9,11,pos,TX,NN,NaN
5,0,9,11,dep,TX,nmod,"((0, 8), 'Aufnahme')"
6,1,0,10,ent,Entlassung,Treatment,NaN
7,1,0,10,pos,Entlassung,NN,NaN
8,1,0,10,dep,Entlassung,root,NaN
9,1,11,13,ent,TX,Treatment,NaN


In [11]:
dfalles.loc[dfalles.loc[dfalles["Zusatz"].apply(lambda x: isinstance(x, str))].index]
dfalles["Zusatz"] = dfalles["Zusatz"].loc[dfalles["Zusatz"].notna()].apply(lambda x: ast.literal_eval(x))
#dfalles["Zusatz"].apply(lambda x: type(x))

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
21,7,203,205,ent,RR syst.,Biological_parameter,"(203, 205)"
73,15,209,211,ent,RR syst.,Biological_parameter,"(209, 211)"
74,15,209,211,ent,RR diast.,Biological_parameter,"(209, 211)"
122,24,67,69,ent,RR syst.,Biological_parameter,"(67, 69)"
257,56,168,176,ent,haeufige Miktionen,Medical_condition,"(168, 176)"
433,99,130,141,ent,diastolisch RR,Biological_parameter,"(130, 141)"
596,148,186,188,ent,RR syst.,Biological_parameter,"(186, 188)"
752,182,175,184,ent,schleicht aus,Treatment,"(175, 184)"
961,216,422,427,ent,Leuko penie,Medical_condition,"(422, 427)"
1090,239,69,74,ent,Hyper Symptome,Medical_condition,"(69, 74)"


In [52]:
ddep = {(snr,st,en):(text) for snr,st,en,text
        in dfalles.loc[:,["SatzNr","StartDok","EndeDok","String"]].values}
dfalles.loc[(dfalles["Typ"]=="dep")&(dfalles["Zusatz"].str.len()==2)]

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
93620,0,9,11,dep,TX,nmod,"(0, 8)"
93622,1,11,13,dep,TX,nmod,"(0, 10)"
93624,2,5,11,dep,soweit,advmod,"(12, 15)"
93625,2,12,15,dep,gut,xcomp,"(0, 4)"
93626,2,16,19,dep,bis,case,"(35, 46)"
93627,2,20,23,dep,auf,case,"(35, 46)"
93628,2,24,34,dep,allgemeine,amod,"(35, 46)"
93629,2,35,46,dep,Schlappheit,obl,"(0, 4)"
93630,2,46,47,dep,.,punct,"(0, 4)"
93631,3,48,51,dep,Vor,case,"(57, 62)"


In [54]:
dfalles["Zusatz"] = dfalles.loc[(dfalles["Typ"]=="dep")&(dfalles["Zusatz"].str.len()==2)].\
    apply(lambda x: (x["Zusatz"],ddep[(x["SatzNr"],)+x["Zusatz"]]),axis=1)
dfalles

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
0,0,0,8,ent,Aufnahme,Treatment,NaN
1,0,9,11,ent,TX,Treatment,NaN
2,1,0,10,ent,Entlassung,Treatment,NaN
3,1,11,13,ent,TX,Treatment,NaN
4,2,0,15,ent,Geht soweit gut,State_of_health,NaN
5,2,24,34,ent,allgemeine,Measurement,NaN
6,2,35,46,ent,Schlappheit,Medical_condition,NaN
7,3,48,62,ent,Vor zwei Tagen,Time_information,NaN
8,3,92,98,ent,Faeden,Medical_device,NaN
9,3,99,107,ent,entfernt,Treatment,NaN


In [62]:
dfalles = dfalles.sort_values(["SatzNr","StartDok"]).drop(columns=["index"]).reset_index(drop=True)

In [64]:
dfalles.to_csv("MACCS_Masterarbeit/oliver/dfalles.csv", sep="\t")

# Alle MACSS-Annotationen

### dfmacss erstellen

In [4]:
from shutil import copyfile
import os
korpuspfad = "/home/olli/Documents/MACCS_Masterarbeit/macss_annotations/"
korpusdatei = "allannocorpus.json"
with open(korpuspfad + korpusdatei) as d:
    korpusdict = json.load(d)

# annotierte Dateien in ein dictionary laden
annodict = {key[key.find("/")+1:]: values for key,values in korpusdict.items()}

In [5]:
#alle relevanten Daten in Arrays laden

# SatzNr 	StartDok 	EndeDok 	Typ 	String 	Wert 	Zusatz

# Entitäten
entities = [
    
    [filename] + sten[0] + ["ent", kat, text] + [sten[1:]]
    if len(sten) > 1 else
    [filename] + sten[0] + ["ent", kat, text, None]
                 
    for filename, content in annodict.items()
     for concept, annotators in content["concepts"].items()
      for kat, text, sten in [ast.literal_eval(concept)]
]

fragments = [
    [filename] + sten + [typ, kat, text, (st,en)]
    for filename, st, en, typ, kat, text, zus in entities
     if zus != None
      for sten in zus
]

# Relationen
relations = [
    [filename, rel1sten[0][0], rel1sten[0][1], "rel", reltyp, txt1] + [rel2sten]

    for filename, content in annodict.items()
     for relation, annotators in content["relations"].items()
      for reltyp, (cat1, txt1, rel1sten), (cat2, txt2, rel2sten) in [ast.literal_eval(relation)]
]

# Attribute
attributes = [
    [filename, sten[0][0], sten[0][1], "att", txt, wert, typ] 
    
    for filename, content in annodict.items()
     for attribute, annotators in content["attributes"].items()
      for typ, wert, (cat, txt, sten) in [ast.literal_eval(attribute)]
    if typ in ['DocTime', 'LevelOfTruth', 'Person']
]

In [7]:
# DataFrame mit allem erstellen
dfmacss = pd.DataFrame(
    entities + fragments + relations + attributes,
    columns=["SatzNr","StartDok","EndeDok","Typ","String","Wert","Zusatz"]
)
dfmacss = dfmacss.sort_values("SatzNr")

# alles speichern ==========================================================================================
#dfmacss.to_csv("MACCS_Masterarbeit/oliver/dfmacss.csv", sep="\t")
dfmacss

,SatzNr,StartDok,EndeDok,Typ,String,Wert,Zusatz
31142,11884_anymous_brat,9,11,ent,Treatment,TX,None
31141,11884_anymous_brat,0,8,ent,Treatment,Aufnahme,None
27386,11885_anymous_brat,11,13,ent,Treatment,TX,None
27385,11885_anymous_brat,0,10,ent,Treatment,Entlassung,None
30991,11886_anymous_brat,413,419,ent,Measurement,kleine,None
30988,11886_anymous_brat,378,392,ent,Local_specification,5 cm unterhalb,None
30989,11886_anymous_brat,397,403,ent,Local_specification,oberen,None
31013,11886_anymous_brat,465,484,ent,Medical_condition,Entzuendungszeichen,None
31012,11886_anymous_brat,448,457,ent,Process,Sekretion,None
31011,11886_anymous_brat,441,447,ent,Measurement,klarer,None


### Sentence-splitting

In [133]:
# POS-Annotation (+ Sentence splitting!) vorbereiten
macsspfad = "/home/olli/Documents/MACCS_Masterarbeit/macss_annotations/data/"
ammerpfad = "/home/olli/Documents/MACCS_Masterarbeit/macss_annotations/pos/Ammers_Pipeline/input/"
standard_unterordner = ["Arztbriefe_preannotated","Verlaufsdaten_preannotated"]
dicannotatoren = {"anna":standard_unterordner,
                  "laura":standard_unterordner,
                  "oliver":standard_unterordner,
                  "michael":["Arztbriefe","Verlaufsdaten"]}

dicfiles = {}
for annotator in dicannotatoren:
    annotatorpfad = macsspfad + annotator + "/"
    for unterordner in dicannotatoren[annotator]:
        finalerpfad = annotatorpfad + unterordner + "/"
        for datei in os.listdir(finalerpfad):
            if datei.endswith(".txt"):
                dicfiles[datei] = finalerpfad+datei

""" was ich korrigieren muss:
&#34 -> '
" -> '
irgendson besonderer white-space ◦ 12361_anymous_brat, 12363_anymous_brat, 12528_anymous_brat, 12548_anymous_brat
no-break whitespace ◦ Brief_0028_brat, Brief_0038_brat
ggf. satzübergreifende Relationen löschen (für Brat)
evtl. falsche Satz-Segmentierung korrigieren:
http://localhost:8888/notebooks/Documents/Konsistenzpr%C3%BCfung.ipynb#Dateien-bearbeiten-und-%C3%BCberschreiben
ggf. komisches Structure-Element löschen und so (für Brat)
http://localhost:8888/notebooks/Documents/OlliKorpusZuBrat.ipynb#Pr%C3%BCfung-vor-letzter-Dependenzanalyse-(ab-10.03.19)
"""
#"""
#Dateien löschen, die vorher darin waren
for datei in os.listdir(ammerpfad):
    os.remove(ammerpfad+datei)

for datei, pfad in dicfiles.items():
    copyfile(pfad, ammerpfad+datei)
#"""

In [152]:
#import subprocess
#subprocess.call(['sed','"s/\&\#34/\'/g"', ammerpfad + "*.txt"])
#subprocess.call(['sed','"s/\"/\'/g"',ammerpfad + "*.txt"])
#das funzt alles nicht

In [6]:
bla = pd.read_csv("/home/olli/Documents/MACCS_Masterarbeit/macss_annotations/Dependenzannotation/clinincal_dependency_parser_ger/input\".conll",
                  sep="\t", quotechar="'")

# Altes BratdateienZuKorpus

In [ ]:
import json
import os
import glob
import re

import plac

CONCEPTS = 'concepts'
RELATIONS = 'relations'
ANNOTATORS = 'annotators'
ATTRIBUTES = 'attributes'
COMMENTS = "comments"

def get_annotions_per_file(file_name):
    """
    extract relations & concepts and their labels, positions & lemmas
    """

    # 1: Read data and extract relations and concepts
    rels = []
    concs = []
    comms = []
    attrs = []
    pattern_r = re.compile("^R\d")
    pattern_t = re.compile("^T\d")
    pattern_c = re.compile("^#\d")
    pattern_a = re.compile("^A\d")

    with open(file_name) as f:
        lines_ = f.readlines()

    for line in lines_:
        lines = re.sub(r"arg(1|2):", "", line)

        if re.match(pattern_r, lines):
            rels.append(lines)
        if re.match(pattern_t, lines):
            concs.append(lines)
        if re.match(pattern_c, lines):
            comms.append(lines)
        if re.match(pattern_a, lines):
            attrs.append(lines)

    # 2: Dictionary with concept id, name, start and end point(s -> fragments), lemma
    conc_dict = {}
    for entry in concs:
        splits = entry.split('\t')
        s2 = splits[1].split(' ', maxsplit=1)
        conc_dict[splits[0]] = s2[0], splits[-1].replace('\n', ''), \
                               tuple([tuple([int(index) for index in s.split()]) for s in s2[1].split(';')])
        #example: 'T1	State_of_health 30 34;42 45	Geht gut'

    # 3: Split relations to get their properties
    relations_short = []
    for entry2 in rels:
        splits2 = entry2.split()
        relations_short.append(splits2)

    # 4: Replace concept-ids with concepts (including all properties)
    relation_label = {}
    for rel_id, rel_name, con1, con2 in relations_short:
        if ':' not in con1 and ':' not in con2:
            relation_label[rel_id] = (rel_name, conc_dict[con1], conc_dict[con2])
        #das mit dem ':' muss überprüft werden wegen veralteter Relationen, die nicht erfasst werden sollen:
        #z.B. R1	negates Neg_Word:T133 Arg:T128	Laura, Brief 0052

    # 5: Sort
    # take min because relations can point backwards
    sorted_relation_list = sorted(relation_label.values(), key=lambda rel: min(rel[1][1][0][0], rel[2][1][0][0]))

    # 6: Dictionary with attribute id, category, attribute and related concept
    attr_dict = {}
    for entry in attrs:
        splits = entry.split('\t')
        s2 = splits[1].split(' ')
        attr_dict[splits[0]] = s2[0], s2[2].replace("\n",""), conc_dict[s2[1]]
        # example: 'A7	DocTime T8 future'

    # 7: Dictionary with comment id, comment and related concept
    comm_dict = {}
    for entry in comms:
        splits = entry.split('\t')
        s2 = splits[1].split(' ')
        if s2[1] != "R14": #there is one single relation that has a comment
            comm_dict[splits[0]] = splits[2].replace("\n",""), conc_dict[s2[1]]
        # example: '#1	AnnotatorNotes T5	String-Variations[floppy (Y), schlapp (Y)], CUI=C0857516; Dict=(MDRGER); STY=(Sign or Symptom);'

    #bla = sorted(list(comm_dict))

    return {RELATIONS: sorted_relation_list,
            CONCEPTS: sorted(list(conc_dict.values()), key=lambda x: x[2][0]),
            ATTRIBUTES: sorted(list(attr_dict.values()), key=lambda x: x[2]),
            COMMENTS: sorted(list(comm_dict.values()), key=lambda x: x[1])}


def process_annotation_files(path, prefix):
    """
    extract the information of all documents of one type
    :param path: path to the document
    :param prefix: type of document(s)
    :return: extracted information of the documents
    """

    print('process files in: %s' % path)
    ann_files = glob.glob(r"%s/*.ann" % path)
    res = {}

    for ann_file in ann_files:
        base_fn = os.path.splitext(os.path.basename(ann_file))[0]
        res['%s/%s' % (prefix, base_fn)] = get_annotions_per_file(ann_file)
    return res


def get_annotations(annotator_path, prefixes=('Verlaufsdaten', 'Arztbriefe')):
    """
    get the annotations of all documents of one annotator
    :param annotator_path: path to the folder of the annotations of the annotator
    :param prefixes: document types
    :return: annotations of documents
    """

    print('process annotation path (%s): %s' % (', '.join(prefixes), annotator_path))
    res = {}

    for prefix in prefixes:
        for subdir in [os.path.join(annotator_path, p) for p in os.listdir(annotator_path) if prefix in p]:
            data = process_annotation_files(subdir, prefix)
            res.update(data)
    return res


def main(path_data: ('path to base folder','option')='data',
         path_out: ('path to output json', 'option')='out.json'):
    """
    merges annotations from the annotators of a brat project with different text types
    assume that all subfolders in data_base_fn have the respective names of th the annotators
    format for every file:
        (ANNOTATORS, CONCEPTS, RELATIONS)
    """
    print("Los")
    all_data = {}

    for annotator in os.listdir(path_data):
        annotator_path = os.path.join(path_data, annotator)
        data = get_annotations(annotator_path)

        for file_key in data:
            current_data = all_data.get(file_key, {ANNOTATORS: [], CONCEPTS: {}, RELATIONS: {}, ATTRIBUTES: {}, COMMENTS: {}})
            current_data[ANNOTATORS].append(annotator)
            current_data[CONCEPTS].update(
                {json.dumps(concept): current_data[CONCEPTS].get(json.dumps(concept), []) + [annotator] for
                 concept in data[file_key][CONCEPTS]})
            current_data[RELATIONS].update(
                {json.dumps(relation): current_data[RELATIONS].get(json.dumps(relation), []) + [annotator] for
                 relation in data[file_key][RELATIONS]})
            current_data[ATTRIBUTES].update(
                {json.dumps(attribute): current_data[ATTRIBUTES].get(json.dumps(attribute), []) + [annotator] for
                 attribute in data[file_key][ATTRIBUTES]})
            current_data[COMMENTS].update(
                {json.dumps(comment): current_data[COMMENTS].get(json.dumps(comment), []) + [annotator] for
                 comment in data[file_key][COMMENTS]})
            all_data[file_key] = current_data

        print('finished %s' % annotator)

    json.dump(all_data, open("allannocorpus.json", 'w'), indent=4)

if __name__ == "__main__":
    plac.call(main)
    print('done')